## Mission Hospital - Package Pricing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sn
import statsmodels.api as smf
from statsmodels.tools.eval_measures import rmse

### Loading the dataset

In [ ]:
mission_df = pd.read_csv( "https://raw.githubusercontent.com/manaranjanp/IIMBClasses/main/regression/hospital.csv" )

In [ ]:
mission_df.head( 10 )

In [ ]:
mission_df.columns

In [ ]:
mission_df.info()

## Exploratory Data Analysis

### Distribution of Total Amount billed to patient

In [ ]:
sn.kdeplot( mission_df.TOTAL_COST_TO_HOSPITAL )

In [ ]:
from scipy import stats

In [ ]:
cost_iqr = stats.iqr( mission_df.TOTAL_COST_TO_HOSPITAL )

In [ ]:
cost_75_percentile = mission_df.TOTAL_COST_TO_HOSPITAL.quantile( 0.75 )

In [ ]:
outliers_costs = cost_75_percentile + 3.0 * cost_iqr

In [ ]:
outliers_costs

In [ ]:
mission_df = mission_df[mission_df.TOTAL_COST_TO_HOSPITAL < outliers_costs]

### Length of stay

In [ ]:
sn.histplot( mission_df.TOTAL_LENGTH_OF_STAY, bins = range(0, 50, 5));

### Find distribution of following parameters

### Correlation between *Age* and *Amount Billed*

In [ ]:
sn.lmplot( x = 'AGE', y = 'TOTAL_COST_TO_HOSPITAL', data = mission_df )

In [ ]:
all_features = ['AGE', 'GENDER', 'MARITAL_STATUS', 
                'KEY_COMPLAINTS_CODE', 'BODY_WEIGHT',
                'BODY_HEIGHT', 'HR_PULSE', 'BP_HIGH', 'BP_LOW', 'RR',
                'PAST_MEDICAL_HISTORY_CODE', 'HB', 'UREA', 
                'CREATININE',
                'MODE_OF_ARRIVAL', 
                'STATE_AT_THE_TIME_OF_ARRIVAL', 
                'TYPE_OF_ADMSN', 'LENGTH_OF_STAY_WARD', 'IMPLANT_USED_']

In [ ]:
continuous_features = ['AGE','BODY_WEIGHT', 'BODY_HEIGHT', 
                       'HR_PULSE', 'BP_HIGH', 'BP_LOW', 'RR', 
                       'HB','UREA', 'CREATININE', 'LENGTH_OF_STAY_WARD', ]

In [ ]:
categorical_features = list( set( all_features ) - set( continuous_features ) )

In [ ]:
categorical_features

## Pairplot with all continuous features

In [ ]:
plt.figure( figsize = (8,6) )
sn.heatmap( mission_df[continuous_features 
                       + ['TOTAL_COST_TO_HOSPITAL']].corr()
           , annot = True
           , cmap = sn.diverging_palette(240, 10));

### Highly Correlated Variables

- BODY_HEIGHT and BODY_WEIGHT seem to be highly correlated. We can remove one of them.
- Or introduce a new variable called BMI = BODY_WEIGHT / BODY_HEIGHT

In [ ]:
continuous_features.remove('BODY_WEIGHT')
continuous_features.remove('BP_HIGH')

### Distribution of total cost for different medical history

In [ ]:
sn.boxplot( y = 'PAST_MEDICAL_HISTORY_CODE', 
           x = 'TOTAL_COST_TO_HOSPITAL', 
           data = mission_df );

### Imputing missing past medical history to None

In [ ]:
mission_final_df = mission_df.copy()

In [ ]:
sn.boxplot( y = 'PAST_MEDICAL_HISTORY_CODE',
           x = 'TOTAL_COST_TO_HOSPITAL', 
           data = mission_df );

In [ ]:
mission_final_df['PAST_MEDICAL_HISTORY_CODE'] = (mission_final_df
                                                 ['PAST_MEDICAL_HISTORY_CODE']
                                                 .fillna( 'None' ))

In [ ]:
sn.boxplot( y = 'PAST_MEDICAL_HISTORY_CODE',
           x = 'TOTAL_COST_TO_HOSPITAL', 
           data = mission_final_df );

### *Mean* Imputation for other features

In [ ]:
mission_final_df["BP_LOW"].fillna(mission_final_df["BP_LOW"].mean(), inplace=True)
mission_final_df["HB"].fillna(mission_final_df["HB"].mean(), inplace=True)
mission_final_df["UREA"].fillna(mission_final_df["UREA"].mean(), inplace=True)
mission_final_df["CREATININE"].fillna(mission_final_df["CREATININE"].mean(), inplace=True)

### Any more null values

In [ ]:
mission_final_df[all_features].info();

In [ ]:
mission_final_df['KEY_COMPLAINTS_CODE'].value_counts()

In [ ]:
mission_final_df['KEY_COMPLAINTS_CODE'] = (mission_final_df['KEY_COMPLAINTS_CODE']
                                           .fillna( 'other-general' ))

In [ ]:
#mission_final_df = mission_final_df.dropna()

In [ ]:
len( mission_final_df )

# Building Linear Regression Model

## Convert Categorical Features to Dummy Variables

In [ ]:
mission_final_df[all_features]

In [ ]:
encoded_mission_final_df = pd.get_dummies(mission_final_df[continuous_features+categorical_features], 
                                          columns = categorical_features,
                                          drop_first = True )

In [ ]:
encoded_mission_final_df[0:5]

In [ ]:
encoded_mission_final_df.columns

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def get_vif_factors( X ):
    X_matrix = X.to_numpy()
    vif = [ variance_inflation_factor( X_matrix, i ) for i in range( X_matrix.shape[1] ) ]
    vif_factors = pd.DataFrame()
    vif_factors['column'] = X.columns
    vif_factors['vif'] = vif
    
    return vif_factors

In [ ]:
vif_factors = get_vif_factors(encoded_mission_final_df )
vif_factors

In [ ]:
high_vif_cols = list(vif_factors[vif_factors.vif > 10]['column'])

In [ ]:
plt.figure( figsize = (8,6) )
sn.heatmap( encoded_mission_final_df[high_vif_cols].corr()
           , annot = True
           , cmap = sn.diverging_palette(240, 10));


In [ ]:
columns_to_remove = ['RR', 'HB', 'MODE_OF_ARRIVAL_WALKED IN', 'HR_PULSE',
                     'PAST_MEDICAL_HISTORY_CODE_None',
                     'BODY_HEIGHT', 'BP_LOW', 'CREATININE']

In [ ]:
x_features = list(encoded_mission_final_df.columns)
x_features = list(set(x_features) - set(columns_to_remove))

In [ ]:
vif_factors = get_vif_factors(encoded_mission_final_df[x_features] )
vif_factors

### Split the datasets into train and test datasets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_features

In [ ]:
Y = mission_final_df['TOTAL_COST_TO_HOSPITAL']
X = smf.add_constant( encoded_mission_final_df[x_features] )

In [ ]:
train_X, test_X, train_y, test_y = train_test_split( X ,
                                                     Y,
                                                    train_size = 0.8,
                                                    random_state = 20)

In [ ]:
len( train_X )

In [ ]:
len( test_y )

In [ ]:
lin = smf.OLS( train_y, train_X )
lm = lin.fit()

In [ ]:
lm.params

In [ ]:
lm.summary2()

In [ ]:
lm.rsquared_adj

## Find Significant Variables

In [ ]:
def get_significant_vars( lm ):
    var_p_vals_df = pd.DataFrame( lm.pvalues )
    var_p_vals_df['vars'] = var_p_vals_df.index
    var_p_vals_df.columns = ['pvals', 'vars']
    return list( var_p_vals_df[var_p_vals_df.pvals <= 0.05]['vars'] )

In [ ]:
significant_vars = get_significant_vars( lm )
significant_vars

## Building the model with significant variables

In [ ]:
best_lreg = smf.OLS( train_y, train_X[significant_vars] )
best_lm = best_lreg.fit()

In [ ]:
best_lm.summary2()

In [ ]:
predict_y = best_lm.predict( test_X[significant_vars] )

### Plot the residuals

In [ ]:
residuals = test_y - predict_y

In [ ]:
plt.scatter( y = residuals, x = test_y );

### Normality Test for residuals

In [ ]:
stats.normaltest( residuals )

#### Note:

The pvalues is less than 0.05, which signifies that the residuals do not follow normal distribution. Null hypothesis is it follows normal distribution.

### PP Plot

In [ ]:
stats.probplot( residuals, dist="norm", plot=plt )
plt.show()

In [ ]:
rmse( test_y, predict_y )

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(test_y, predict_y)

## Lasso Model

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso = Lasso(alpha = 1000.0)

In [ ]:
lasso.fit(train_X[x_features], train_y)

In [ ]:
lasso.score(test_X[x_features], test_y)

In [ ]:
lasso.intercept_

In [ ]:
lasso.coef_

In [ ]:
lasso_coefs_df = pd.DataFrame({ "features" : x_features, 
                                "coefs" : lasso.coef_})

In [ ]:
lasso_coefs_df[lasso_coefs_df.coefs != 0.0]

In [ ]:
significant_vars

## KNN Model

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
knn_v1 = KNeighborsRegressor(n_neighbors = 8, weights = 'distance')

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(train_X[x_features])

In [ ]:
train_X_scaled = scaler.transform(train_X[x_features])
test_X_scaled = scaler.transform(test_X[x_features])

In [ ]:
knn_v1.fit(train_X_scaled, train_y)

In [ ]:
knn_pred_y = knn_v1.predict(test_X_scaled)

In [ ]:
r2_score(test_y, knn_pred_y)